# U.S. Trade data

In [119]:
import pandas as pd
from urllib.request import urlopen 
import os
import glob
import requests
import matplotlib
import numpy as np
from altair import datum
import altair as alt
import altair_latimes as lat
from sodapy import Socrata
import cpi
alt.themes.register('latimes', lat.theme)
alt.themes.enable('latimes')
pd.options.display.max_columns = 50
pd.options.display.max_rows = 34000

In [120]:
# cpi.update()

In [121]:
#https://usatrade.census.gov/data/Perspective60/View/dispview.aspx?ReportId=132782
ca_port_imports = pd.read_csv('input/trade/CA port imports.csv', skiprows=4)

In [122]:
ca_port_imports = ca_port_imports.drop(['Unnamed: 3'], axis=1)\
    .rename(columns={'Time':'month', 'Port':'port', \
                     'Customs Value (Gen) ($US)':'value'})

In [123]:
ca_port_imports.value = ca_port_imports.value.str.replace(',', '', regex=False)

In [124]:
ca_port_imports['port'] = ca_port_imports['port'].str.replace(', CA (District)', '', regex=False)

In [125]:
ca_port_imports['date'] = pd.to_datetime(ca_port_imports['month'])

In [161]:
ca_port_imports['year'] = pd.to_datetime(ca_port_imports['year'])

In [162]:
ca_port_imports['year'] = ca_port_imports['date'].dt.year

In [163]:
ca_port_imports['value'] = ca_port_imports['value'].astype(int)

In [164]:
ca_port_imports.dtypes

month                     object
port                      object
value                      int64
date              datetime64[ns]
year                       int64
adjusted_value           float64
dtype: object

In [165]:
ca_port_imports['adjusted_value'] = ca_port_imports.apply(lambda x: cpi.inflate(x.value, x.year), axis=1)

CPIObjectDoesNotExist: Index of annual type for 2020-01-01 does not exist

In [144]:
ca_port_imports_group = ca_port_imports.groupby(['year']).agg('sum').reset_index()

In [146]:
ca_port_imports_pivot = ca_port_imports.pivot(index=['date', 'month'], columns='port', values='adjusted_value').reset_index()

In [147]:
ca_port_imports_pivot = pd.DataFrame(ca_port_imports_pivot[~ca_port_imports_pivot['month'].str.contains('December ')])

In [148]:
ca_port_imports_pivot['total'] =\
    ca_port_imports_pivot.apply(lambda x: x['Los Angeles'] + x['San Diego'] + x['San Francisco'], axis=1)

In [149]:
ca_port_imports_pivot.tail()

port,date,month,Los Angeles,San Diego,San Francisco,total
210,2020-07-01,July 2020,NaN,NaN,NaN,NaN
211,2020-08-01,August 2020,NaN,NaN,NaN,NaN
212,2020-09-01,September 2020,NaN,NaN,NaN,NaN
213,2020-10-01,October 2020,NaN,NaN,NaN,NaN
214,2020-11-01,November 2020,NaN,NaN,NaN,NaN


In [150]:
ca_port_imports_group

,year,value,adjusted_value
0,2003,292952380837,4.070398e+11
1,2004,334002441802,4.520384e+11
2,2005,362897234122,4.750498e+11
3,2006,414329841177,5.254282e+11
4,2007,428873831118,5.288104e+11
5,2008,430436610282,5.111129e+11
6,2009,334006155500,3.980246e+11
7,2010,405120886727,4.749789e+11
8,2011,451859405364,5.135660e+11
9,2012,473832641534,5.276211e+11


In [151]:
alt.Chart(ca_port_imports_group).mark_bar().encode(
    x=alt.X('year:N', title=' ', axis=alt.Axis(format='')),
    y=alt.Y('adjusted_value:Q', title=' ', axis=alt.Axis(tickSize=0,domainOpacity=0,format='',\
                                               tickCount=5,offset=3, gridWidth=.6, gridColor='#dddddd'))
).configure_view(strokeOpacity=0).properties(width=640, height=400,
    title='Imports to California ports, January-March, by year'
).configure_legend(
    orient='top',
    symbolType='square'
).configure_axis(
    labelFontSize=12)

alt.Chart(...)

---

### Exports data from U.S. Trade Online

#### All state-level exports

In [16]:
#https://usatrade.census.gov/data/Perspective60/View/dispview.aspx?ReportId=132783
caexports = pd.read_csv('input/trade/CA Exports by HS Commodities.csv', skiprows=4)

FileNotFoundError: [Errno 2] No such file or directory: 'input/trade/CA Exports by HS Commodities.csv'

In [ ]:
caexports = caexports.drop(['Unnamed: 3'], axis=1)\
    .rename(columns={'Time':'month', 'State':'state', \
                     'Total Value ($US)':'value'})

In [ ]:
caexports.head(30)

#### Exports from CA ports

In [68]:
#https://usatrade.census.gov/data/Perspective60/View/dispview.aspx?ReportId=132781
ca_port_exports = pd.read_csv('input/trade/CA port exports.csv', skiprows=4)

In [69]:
ca_port_exports = ca_port_exports.drop(['Unnamed: 3'], axis=1)\
    .rename(columns={'Time':'month', 'Port':'port', \
                     'Total Exports Value ($US)':'value'})

In [70]:
ca_port_exports.value = ca_port_exports.value.str.replace(',', '', regex=False)

In [71]:
ca_port_exports['port'] = ca_port_exports['port'].str.replace(', CA (District)', '', regex=False)

In [72]:
ca_port_exports['date'] = pd.to_datetime(ca_port_exports['month'])

In [73]:
ca_port_exports_pivot = ca_port_exports.pivot(index='date', columns='port', values='value').reset_index()

In [74]:
ca_port_exports_pivot.tail(20)

port,date,Los Angeles,San Diego,San Francisco
195,2019-04-01,10201524473,2199599791,4566297470
196,2019-05-01,10886096975,2232870751,5435556329
197,2019-06-01,10435975136,2051173810,4392320657
198,2019-07-01,9592006213,2190145421,4635366065
199,2019-08-01,9955460952,2189746243,5224181216
200,2019-09-01,9558148434,2011033936,4414598480
201,2019-10-01,10515573571,2316213803,5453555203
202,2019-11-01,10140525490,1985478208,5654123417
203,2019-12-01,10105092093,1668168374,4930322419
204,2020-01-01,8837155410,2042073177,5033882619


In [75]:
ca_port_exports.tail(20)

,month,port,value,date
625,May 2020,San Diego,1304698199,2020-05-01
626,May 2020,San Francisco,4638092080,2020-05-01
627,June 2020,Los Angeles,8306329975,2020-06-01
628,June 2020,San Diego,1746325472,2020-06-01
629,June 2020,San Francisco,4385669192,2020-06-01
630,July 2020,Los Angeles,8628762730,2020-07-01
631,July 2020,San Diego,1863108826,2020-07-01
632,July 2020,San Francisco,5266720403,2020-07-01
633,August 2020,Los Angeles,8991453137,2020-08-01
634,August 2020,San Diego,1825585067,2020-08-01


In [76]:
# alt.Chart(ca_port_exports).mark_bar().encode(
#     x=alt.X('month:N'),
#     y=alt.Y('value:Q'),
#     color='port:N'
# )